<a href="https://colab.research.google.com/github/bluetinue/sharehouse/blob/main/CountryName.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 人名分类器


In [50]:
#@title 导包
# prompt: 装载谷歌硬盘
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
# 显示所有变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 导入torch工具
import torch
# 导入nn准备构建模型
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# 导入torch的数据源 数据迭代器工具包
from  torch.utils.data import Dataset, DataLoader
# 用于获得常见字母及字符规范化
import string
# 导入时间工具包
import time
# 引入制图工具包
import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# @title 读取数据
data = pd.read_csv('/content/drive/MyDrive/NLP/data/name_classfication.txt',sep='\t')

In [22]:
# @title 数据预处理

#去重后的词表
all_letters = string.ascii_letters + " ,.:'"
n_letters = len(all_letters)
# 国家名 种类数
categorys = ['Italian', 'English', 'Arabic', 'Spanish', 'Scottish', 'Irish', 'Chinese', 'Vietnamese', 'Japanese',
             'French', 'Greek', 'Dutch', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Czech', 'German']
# 国家名 个数
categorynum = len(categorys)
print('categorys--->', categorys)
print('all_letters--->', all_letters)

# 读取数据
def read_data(path):
  x,y = [],[]
  with open(path,'r') as f:
    lines = f.readlines()
    for line in lines:
      line = line.strip().split('\t')
      x.append(line[0])
      y.append(line[1])
  return x,y

categorys---> ['Italian', 'English', 'Arabic', 'Spanish', 'Scottish', 'Irish', 'Chinese', 'Vietnamese', 'Japanese', 'French', 'Greek', 'Dutch', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Czech', 'German']
all_letters---> abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ,.:'


In [53]:
#@title 数据加载器
#定义数据封装器类
class NameclassifcationDataset(Dataset):
    def __init__(self,x,y):
      #将读取的数据封装起来
        self.x = x
        self.y = y
        self.sample_len = len(x)

    #调用len方法返回x的数量
    def __len__(self):
        return self.sample_len

    #对进行切片操作时调用
    def __getitem__(self, idx):
      idx = min(max(0,idx),self.sample_len-1)

      x = self.x[idx]
      y = self.y[idx]

      tensor_x = torch.zeros(len(x),n_letters)
      for i,v in enumerate(x):
        tensor_x[i][all_letters.find(v)] = 1

      tensor_y = torch.tensor(categorys.index(y))

      return tensor_x,tensor_y

In [64]:
#@title 迭代器
def get_traindata():
  path = "/content/drive/MyDrive/NLP/data/name_classfication.txt"
  x,y = read_data(path)
  nameclassdataset = NameclassifcationDataset(x,y)
  train_loader = DataLoader(nameclassdataset,batch_size=1,shuffle=True)
  for x,y in train_loader:
    print(x.shape,y.shape)
    break
  return train_loader

In [72]:
#@title 构建RNN
# RNN类 实现思路分析：
# 1 init函数 准备三个层 self.rnn self.linear self.softmax=nn.LogSoftmax(dim=-1)
#    def __init__(self, input_size, hidden_size, output_size, num_layers=1)

# 2 forward(input, hidden)函数
#   让数据经过三个层 返回softmax结果和hn
#   形状变化 [seqlen,1,57],[1,1,128]) -> [seqlen,1,128],[1,1,128]

# 3 初始化隐藏层输入数据 inithidden()
#   形状[self.num_layers, 1, self.hidden_size]
class myRNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, num_layers=1):
    super(myRNN, self).__init__()
    self.input_size = input_size
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.output_size = output_size

    self.rnn = nn.RNN(input_size, hidden_size, num_layers)
    self.linear = nn.Linear(hidden_size, output_size)
    #根据后边的损失函数如果是CrossEntropyLoss,就不需要softmax，是NLLLoss就需要加上softmax
    self.softmax = nn.LogSoftmax(dim=-1)

  #前向传播
  def forward(self,input,hidden):
    #input

In [ ]:
#@title 构建LSTM

In [ ]:
#@title 构建GRU

In [71]:
#@title 训练
myrnn = myRNN(input_size=n_letters, hidden_size=128, output_size=categorynum)
myrnn

myRNN(
  (rnn): RNN(57, 128)
  (linear): Linear(in_features=128, out_features=18, bias=True)
  (softmax): LogSoftmax(dim=-1)
)